In [2]:
import os
import azureml.core
from azureml.core import Workspace, Experiment, Environment
from azureml.core.conda_dependencies import CondaDependencies

from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule

from azureml.core.runconfig import RunConfiguration




In [5]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

ads-demo-aml
Demos-ADS
eastus
f5052c6f-7974-48eb-b0ba-f390a5c40ab8


In [9]:
aml_compute_target = "hw-compute"

try: #Look for compute cluster
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("found existing compute target.")
except ComputeTargetException: #Create compute cluster if not exists.
    print("creating new compute target")
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 0, 
                                                                max_nodes = 4)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached")

creating new compute target
InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Azure Machine Learning Compute attached


In [10]:
aml_compute.add_identity('SystemAssigned')

In [ ]:
env = Environment(name='myinlineenv')

conda_dep = CondaDependencies()
conda_dep.add_conda_package("pyodbc")

env.python.conda_dependencies=conda_dep

In [8]:
aml_run_config = RunConfiguration()
aml_run_config.target = aml_compute_target
aml_run_config.environment = env

NameError: name 'env' is not defined

In [ ]:
aml_run_config

{
    "script": null,
    "arguments": [],
    "target": "hw-compute",
    "framework": "Python",
    "communicator": "None",
    "maxRunDurationSeconds": null,
    "nodeCount": 1,
    "priority": null,
    "environment": {
        "name": "myinlineenv",
        "version": null,
        "environmentVariables": {
            "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
        },
        "python": {
            "userManagedDependencies": false,
            "interpreterPath": "python",
            "condaDependenciesFile": null,
            "baseCondaEnvironment": null,
            "condaDependencies": {
                "name": "project_environment",
                "dependencies": [
                    "python=3.6.2",
                    {
                        "pip": [
                            "azureml-defaults"
                        ]
                    },
                    "pyodbc"
                ],
                "channels": [
                    "anaconda",
                    "co

In [ ]:
source_directory = './'
print('Source directory for the step is {}.'.format(os.path.realpath(source_directory)))

step1 = PythonScriptStep(name="library_step",
                         script_name="iroc_library_test.py", 
                         compute_target=aml_compute,
                         runconfig = aml_run_config,
                         source_directory=source_directory,
                         allow_reuse=False)
print("Step1 created")

Source directory for the step is /mnt/batch/tasks/shared/LS_root/mounts/clusters/jodobrze1/code/JaceTest.
Step1 created


In [ ]:
steps = [step1]

pipeline1 = Pipeline(workspace=ws, steps=steps)
print ("Pipeline is built")

Pipeline is built


In [ ]:
pipeline1.validate()
print("Pipeline validation complete")

Step library_step is ready to be created [3661e29d]
Pipeline validation complete


In [ ]:
#Optional experiment run.
pipeline_run1 = Experiment(ws, 'iroc_library1').submit(pipeline1, regenerate_outputs=False)
print("Pipeline is submitted for execution")

Created step library_step [3661e29d][bc39a38a-f5e1-4cd1-b830-9cae8c1ab7d3], (This step will run and generate new outputs)
Submitted PipelineRun f46f345a-67af-46f1-b3d3-ed4e4425907f
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/f46f345a-67af-46f1-b3d3-ed4e4425907f?wsid=/subscriptions/f5052c6f-7974-48eb-b0ba-f390a5c40ab8/resourcegroups/demos-ads/workspaces/ads-demo-aml&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
Pipeline is submitted for execution


In [ ]:
pipeline1.publish(name='iroc_library1_pipeline')

Name,Id,Status,Endpoint
iroc_library1_pipeline,aba824d3-7532-4de5-8391-ec9738aa16b2,Active,REST Endpoint


In [ ]:
keyvault = ws.get_default_keyvault()

print(keyvault)

In [ ]:
keyvault.set_secret(name="connstring", value = "hello world")